<a href="https://colab.research.google.com/github/JairEsc/Mat_Apl/blob/main/MN_T10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def check_diag(A,tol,pr=False):#Pensado para matrices simetricas
    boolean=True
    breaker=False
    for i in range(len(A)):
        for j in range(i,len(A)):
            if(i!=j):
                if(abs(A[i][j])>tol):
                    boolean=False;
                    breaker=True;
                    break;
        if(breaker==True):
            break
    return(boolean)
def entrada_max(A):#Regresa una lista [i,j], tal que a_{i,j} es el valor maximo en valor absoluto.
#Pensado para matrices simetricas
    max=abs(A[0][1])
    index=[0,1]
    for i in range(len(A)):
        for j in range(i,len(A[0])):
            if(i!=j):
                if(max<abs(A[i][j])):
                    max=abs(A[i][j])
                    index=i,j
    return(index)
import numpy as np


In [2]:
def Jacobi_rot(A,tol,max_Iter):
    count=0
    n=len(A)
    P=np.identity(n)
    while((check_diag(A,tol)==False)&(count<=max_Iter)):
        p,q=entrada_max(A)
        a=A[p][q]
        if(A[p][p]==A[q][q]):
            theta=np.pi/4
        else:
            theta=0.5*np.arctan(2*a/(A[p][p]-A[q][q]))
        #Ya tenemos la rotacion y podemos hacer el producto de matrices
        R=np.identity(n)
        R[p][p]=np.cos(theta)
        R[p][q]=-np.sin(theta)
        R[q][p]=-R[p][q]
        R[q][q]=R[p][p]
        A=np.dot(R.T,np.dot(A,R))#Se puede mejorar, modificando solo aquellas entradas que cambian.
        count+=1
        P=np.dot(P,R)
    if(count>max_Iter):
        print("Maxima iteracion alcanzada")
    else:
        print("num. Iteraciones: ", str(count))
    return(P,np.diagonal(A))

In [3]:
A_1=np.loadtxt("Eigen_3x3.txt",skiprows=1);
print("Matriz 3x3:\n")
print(A_1)
P,D=Jacobi_rot(A_1,10**(-5),max_Iter=1000)
A_1=np.loadtxt("Eigen_3x3.txt",skiprows=1);
print("\nVerificación:    A=P*D*P.T\n")
print(np.dot(np.dot(P,np.diag(D)),P.T))

OSError: ignored

In [ ]:
A_1=np.loadtxt("Eigen_50x50.txt",skiprows=1);
print("Matriz 50x50:\n")
#print(A_1)
P,D=Jacobi_rot(A_1,10**(-5),max_Iter=10000)
A_1=np.loadtxt("Eigen_50x50.txt",skiprows=1);
print("Maxima diferencia entre mi resultado y la matriz original:\n")
print(np.max(A_1-np.dot(np.dot(P,np.diag(D)),P.T)))

In [ ]:
A_1=np.loadtxt("Eigen_125x125.txt",skiprows=1);
#print(A_1)
print("Matriz 125x125:\n")
P,D=Jacobi_rot(A_1,10**(-5),max_Iter=1000)
A_1=np.loadtxt("Eigen_125x125.txt",skiprows=1);
print("Maxima diferencia entre mi resultado y la matriz original:\n")
print(np.max(A_1-np.dot(np.dot(P,np.diag(D)),P.T)))

In [18]:
import numpy as np
import math
def max_eigen(A,tol,max_iter,pr=True,v_0=[]):
    n=len(A)
    if(len(v_0)==0):#Es opcional
        v_0=(np.zeros(n)+1/math.sqrt(n))#Normalizado
    v_1=np.dot(A,v_0)#iteracion
    lambda_0=np.dot(v_0,v_1)
    lambda_1=np.sqrt(np.dot(v_1,v_1))#se toma raíz para poder compararlos
    count=0
    while((abs(lambda_0-lambda_1)>tol)&(count<max_iter)):
        count+=1
        v_0=v_1/np.linalg.norm(v_1)#Normalizacion
        v_1=np.dot(A,v_0)#iteracion
        lambda_0=lambda_1
        lambda_1=np.dot(v_0,v_1)
    if(count>max_iter):
        return("Maxima iteracion alcanzada")
    else:
        if(pr==True):
            print("numero de iteraciones: "+str(count))
        return([lambda_1,v_1])
def min_eigen(A,tol,max_iter,pr=True,v_0=[]):
    n=len(A)
    if(len(v_0)==0):#Es opcional
        v_0=(np.zeros(n)+1/math.sqrt(n))#Normalizado
    L,U=factorizar_LU(A)
    v_1=solve(L,U,v_0)#Se resuelve el sistema
    lambda_0=np.dot(v_0,v_1)
    lambda_1=np.sqrt(np.dot(v_1,v_1))#se toma raíz para poder compararlos
    count=0
    while((abs(lambda_0-lambda_1)>tol)&(count<max_iter)):
        count+=1
        v_0=v_1/np.linalg.norm(v_1)#Se normaliza
        v_1=solve(L_m=L,U_m=U,b=v_0)#iteracion
        lambda_0=lambda_1
        lambda_1=np.dot(v_0,v_1)
    if(count>=max_iter):
        return("Maxima iteracion alcanzada")
    else:
        if(pr==True):
            print("numero de iteraciones: "+str(count))
        return([1/lambda_1,v_1])
def deflacion(A,l,v):
    for i in range(len(A)):
        for j in range(len(A[0])):
            A[i][j]-=l*v[i]*v[j]
    return(A)
def shift(A,alpha):
    B=np.copy(A)
    for i in range(len(A)):
        B[i][i]-=alpha
    return B
import numpy as np
def factorizar_LU(matriz):#Codigo reciclado: Factorizacion LU
    n=len(matriz)
    L=np.zeros((n,n))
    U=np.zeros((n,n))
    for j in range(n):
        for i in range(j,n):
            sum=0
            for k in range(i):
                sum+=L[i][k]*U[k][j]
            L[i][j]=matriz[i][j]-sum
            if(i>j):
                sum=0
                for k in range(j):
                    sum+=L[j][k]*U[k][i]
                U[j][i]=(matriz[j][i]-sum)/L[j][j]
            else: 
                U[i][j]=1 
                if(L[i][i]==0):
                    print("Error en la construcción de L")
    return([L,U])

def superior(list,b):#Codigo reciclado:Resolver sistema triangular
  n=len(list)
  X=[b[n-1]/list[n-1][n-1]]
  X_inv=[X[0]]
  for i in range(1,n):#len(list) es el número de renglones de la matriz
    acum=0
    for j in range(i):#0<=j<=i-1 
      acum+=X[j]*list[n-i-1][n-1-j]
    X.append((b[n-1-i]-acum)/list[n-1-i][n-1-i])
    X_inv.insert(0,X[i])
  return X_inv

#Codigo reciclado:Resolver sistema triangular
def inferior(list,b):#list debe contener las entradas de la matriz triangular.
#Será aceptable introducir un arreglo de arreglos de distintos tamaños, para solo introducir las entradas no cero.
  X=[b[0]/list[0][0]]
  for i in range(1,len(list)):#len(list) es el número de renglones de la matriz
    acum=0
    for j in range(i):#0<=j<=i-1 
      acum+=X[j]*list[i][j]
    X.append((b[i]-acum)/list[i][i])
  return X
def solve(L_m,U_m,b):
    y=inferior(L_m,b)#Resolver Ly=b
    return(superior(U_m,y))#Resolver Ux=y

In [28]:
import numpy as np
def multiple_min_eigen(A,m,tol,max_iter,pr=False,v_0=[]):
    n=len(A)
    B=np.copy(A)
    if(len(v_0)==0):#Es opcional
        v_0=(np.zeros(n)+1/np.sqrt(n))#Normalizado
    approximations=np.diag(B)
    approximations=np.sort(approximations)
    eigenvalues=np.zeros(m)
    eigenvectors=[]
    iter_counts=np.zeros(m)
    #print(approximations)
    for j in range(0,m):
        v_0=(np.zeros(n)+1/np.sqrt(n))#Normalizado
        alpha=approximations[j]+np.random.uniform(-0.2,0.2)
        A=shift(B,alpha)
        L,U=factorizar_LU(A)
        ########## Potencia inversa.
        v_1=solve(L,U,v_0)#Se resuelve el sistema
        lambda_0=np.dot(v_0,v_1)
        lambda_1=np.sqrt(np.dot(v_1,v_1))#se toma raíz para poder compararlos
        count=0
        while((abs(lambda_0-lambda_1)>tol)&(count<max_iter)):
            count+=1
            v_0=v_1/np.linalg.norm(v_1)#Se normaliza
            v_1=solve(L_m=L,U_m=U,b=v_0)#iteracion
            lambda_0=lambda_1
            lambda_1=np.dot(v_0,v_1)
        eigenvalues[j]=(1/lambda_1)+alpha
        eigenvectors.append(v_1/np.linalg.norm(v_1))
        ###########
        iter_counts[j]=count
    if(pr==True):
        print("iteraciones: ", iter_counts)
    return(eigenvalues,eigenvectors)

In [49]:
A=np.loadtxt("Eigen_3x3.txt",skiprows=1)
print("matriz 3x3:\n")
print(A)
eigenpairs=multiple_min_eigen(A,m=2,tol=10**(-6),max_iter=1000,pr=True)
errores=[]
print("\nEigenvalores: ", eigenpairs[0])
for j in range(2):
    errores.append(np.linalg.norm(np.dot(A,eigenpairs[1][j])-eigenpairs[0][j]*eigenpairs[1][j]))
print("\nError maximo: ||A*v-\lambda*v||^2: ", np.max(errores) )

matriz 3x3:

[[ 3.  -0.1 -0.2]
 [-0.1  7.  -0.3]
 [-0.2 -0.3 10. ]]
iteraciones:  [4. 4.]

Eigenvalores:  [2.99134332 6.97386041]

Error maximo: ||A*v-\lambda*v||^2:  3.407274821313779e-07


In [50]:
A=np.loadtxt("Eigen_50x50.txt",skiprows=1)
print("matriz 50x50:\n")
eigenpairs=multiple_min_eigen(A,m=45,tol=10**(-6),max_iter=1000,pr=True)
errores=[]
print("Eigenvalores: ", eigenpairs[0])
for j in range(45):
    errores.append(np.linalg.norm(np.dot(A,eigenpairs[1][j])-eigenpairs[0][j]*eigenpairs[1][j]))
print("\nError maximo: ||A*v-\lambda*v||^2: ", np.max(errores) )

matriz 50x50:

iteraciones:  [3. 3. 3. 4. 3. 3. 3. 3. 3. 3. 3. 3. 3. 4. 3. 4. 3. 4. 3. 3. 3. 3. 3. 3.
 4. 3. 3. 3. 3. 3. 4. 3. 4. 3. 3. 3. 3. 4. 4. 3. 3. 4. 3. 3. 3.]
Eigenvalores:  [  9.99805017  19.99879386  29.99893095  39.99976329  49.99836638
  59.99984468  69.99913545  79.99973777  89.99986381  99.99943662
 109.99971179 119.99983852 129.9985409  140.00024045 149.99914125
 159.99932191 170.00050011 179.9991979  190.00074894 199.99988997
 209.99915675 220.00097139 229.99959541 240.00015469 250.00004245
 260.00003911 269.99975585 279.99994991 289.99969859 300.00094069
 309.99958264 320.00114132 329.99916406 340.00013986 349.99992858
 360.00107749 369.99916481 380.00121039 390.00094305 400.00081716
 410.00044384 419.99982224 430.00047518 440.0004189  450.00048327]

Error maximo: ||A*v-\lambda*v||^2:  1.0712728425219974e-06


In [36]:
A=np.loadtxt("Eigen_1000x1000.txt",skiprows=1)
eigenpairs=multiple_min_eigen(A,m=1,tol=10**(-6),max_iter=1000,pr=True)


iteraciones:  [3.]


In [26]:
A=np.loadtxt("Eigen_50x50.txt",skiprows=1)
print(np.linalg.norm(np.dot(A,eigenpairs[1][1])-eigenpairs[0][1]*eigenpairs[1][1]))

8.358859302903426e-07
